In [7]:
import glob
import subprocess
from tqdm import tqdm
import multiprocessing
from time import sleep
import random

def gulp(input_path):
    sleep(random.random())
    run = True
    output_path = input_path[:-3] + 'out'
    try:
        with open(output_path) as f:
            file = f.read()
        if 'Job Finished at' in file:
            run = False
    except:
        pass
    if run:
        with open(input_path) as f:
            output_path = input_path[:-3] + 'out'
            with open(output_path, 'w') as o:
                try:
                    a = subprocess.run(['gulp'], stdin = f, stdout = o, timeout = 900)
                except:
                    pass

all_cases = sorted(glob.glob('*.inp'), key = lambda x: x.split('/')[-1].split('.')[0])

complete = []

for case in all_cases:
    try:
        with open(case[:-3] + 'out', 'r') as f:
            g = f.read()
        if 'Final energy' in g:
            complete.append(case)
    except:
        pass

cases = [case for case in all_cases if case not in complete]

N = len(cases)

def update():
    pbar.update(1)

if __name__ == '__main__':
    with multiprocessing.Pool() as pool:
        for result in tqdm(pool.imap_unordered(gulp, cases), total=len(cases)):
            pass
    pool.close()
    pool.join()

100%|██████████| 1/1 [00:11<00:00, 11.27s/it]


In [12]:
import re
import glob

def getEnergy(path):
    with open(path) as file:
        content = file.readlines()
    for line in content:
        if "Final defect energy" in line:
            return float(re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", line)[0])

def getLatticeEnergy(path):
    with open(path) as file:
        content = file.readlines()
    for line in content:
        if "Final energy" in line:
            return float(re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", line)[0])

def energyList(data):
    files = glob.glob(data)
    energies = dict()
    for f in files:
        energies[f] = getEnergy(f)
    return energies
        
def latticeEnergy(data):
    lattice_energy = getLatticeEnergy(data)
    return lattice_energy

energies = energyList('*.out')
lattice_energy = latticeEnergy('na3ocl_na_vac.out')

na2o = latticeEnergy('ana2o.out') #lattice parameter fromm chemtube3d.com (fluorite)
nacl = latticeEnergy('anacl.out') #lattice parameter from worksheet (rocksalt)
mgo = latticeEnergy('amgo.out') #lattice parameter from crystec.de (rocksalt)
mgcl2 = latticeEnergy('amgcl2.out') #lattice parameter from materialsproject.org
cao = latticeEnergy('acao.out') #lattice parameter from ucl.ac.uk (rocksalt)
cacl2 = latticeEnergy('acacl2.out') / 2 #lattice parameter from materialsproject.org
sro = latticeEnergy('asro.out') #lattice parameter from ucl.ac.uk (rocksalt)
srcl2 = latticeEnergy('asrcl2.out') #lattice parameter from materialsproject.org (fluorite)
bao = latticeEnergy('abao.out') #lattice parameter from ucl.ac.uk (rocksalt)
bacl2 = latticeEnergy('abacl2.out') #lattice parameter from materialsproject.org (fluorite)

magic = open('/home/ben/Documents/gulp_calcs/0_summary/output_na3ocl_khandy.txt', 'w+')

magic.write("Na vacancy defect energy: " + str(round(energies['na3ocl_na_vac.out'], 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Na vacancy migration barrier: " + str(round((energies['na3ocl_na_vmig.out'] - energies['na3ocl_na_vac.out']), 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Na interstitial defect energy: " + str(round(energies['na3ocl_na_int.out'], 2)) + " eV" + "\n")
magic.write("\n")
magic.write("O vacancy defect energy: " + str(round(energies['na3ocl_o_vac.out'], 2)) + " eV" + "\n")
magic.write("\n")
magic.write("O interstitial defect energy: " + str(round(energies['na3ocl_o_int.out'], 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Cl vacancy defect energy: " + str(round(energies['na3ocl_cl_vac.out'], 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Cl interstitial defect energy: " + str(round(energies['na3ocl_cl_int.out'], 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Na3OCl Schottky defect energy: " + str(round((energies['na3ocl_na_vac.out'] * 3 + energies['na3ocl_o_vac.out'] + energies['na3ocl_cl_vac.out'] + lattice_energy), 2)) + " eV" + "\n")
magic.write("\n")
magic.write("NaCl partial Schottky defect energy: " + str(round((energies['na3ocl_na_vac.out'] + energies['na3ocl_cl_vac.out'] + nacl), 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Na2O partial Schottky defect energy: " + str(round((energies['na3ocl_na_vac.out'] * 2 + energies['na3ocl_o_vac.out'] + na2o), 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Na Frenkel defect energy: " + str(round((energies['na3ocl_na_vac.out'] + energies['na3ocl_na_int.out']), 2)) + " eV" + "\n")
magic.write("\n")
magic.write("O Frenkel defect energy: " + str(round((energies['na3ocl_o_vac.out'] + energies['na3ocl_o_int.out']), 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Cl Frenkel defect energy: " + str(round((energies['na3ocl_cl_vac.out'] + energies['na3ocl_cl_int.out']), 2)) + " eV" + "\n")
magic.write("\n")
magic.write("\n")
magic.write("Lattice energy of Na3OCl (from calculations): " + str(round(lattice_energy, 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Lattice energy of NaCl (from calculations): " + str(round(nacl, 2)) + " eV" + "\n")
magic.write("\n")
magic.write("Lattice energy of Na2O (from calculations): " + str(round(na2o, 2)) + " eV" + "\n")

magic.close()